# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *
# Set your preferred name
USER = "Lisa"
# Set the data and output directories
DATA_ROOT = '/Users/wjt/Desktop/is_data'
OUTPUT_ROOT = '/Users/wjt/Desktop/is_data/v1'
TRAIT = 'Underweight'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [9]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/wjt/Desktop/is_data/TCGA'

If no match is found, jump directly to GEO in Part 2.2

In [7]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [8]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [9]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [10]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [11]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['AJCC_Stage_nature2012', 'Age_at_Initial_Pathologic_Diagnosis_nature2012', 'CN_Clusters_nature2012', 'Converted_Stage_nature2012', 'Days_to_Date_of_Last_Contact_nature2012', 'Days_to_date_of_Death_nature2012', 'ER_Status_nature2012', 'Gender_nature2012', 'HER2_Final_Status_nature2012', 'Integrated_Clusters_no_exp__nature2012', 'Integrated_Clusters_unsup_exp__nature2012', 'Integrated_Clusters_with_PAM50__nature2012', 'Metastasis_Coded_nature2012', 'Metastasis_nature2012', '

In [12]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'Age_at_Initial_Pathologic_Diagnosis_nature2012': [nan, nan, nan, nan, nan],
 'age_at_initial_pathologic_diagnosis': [55.0, 50.0, 62.0, 52.0, 50.0],
 'days_to_birth': [-20211.0, -18538.0, -22848.0, -19074.0, -18371.0],
 'year_of_initial_pathologic_diagnosis': [2004.0,
  2003.0,
  2011.0,
  2011.0,
  2013.0]}

In [14]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [15]:
preview_df(clinical_data[candidate_gender_cols])

{'Gender_nature2012': [nan, nan, nan, nan, nan],
 'gender': ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']}

In [16]:
gender_col = 'gender'

In [17]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [18]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [19]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Epilepsy,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-3C-AAAU-01,1,55.0,0.0,0.607308,-3.194126,-0.531035,-0.145872,0.237422,-0.29921,-0.142694,...,-0.748878,0.565583,-0.767233,-1.28139,-0.271377,-0.492286,0.360373,1.067905,0.076267,-0.392212
TCGA-3C-AALI-01,1,50.0,0.0,-0.641192,-4.928226,0.095465,0.098128,-0.541978,-0.32291,-0.044694,...,0.312922,0.079683,0.475267,1.99661,0.161423,0.273714,0.644673,-0.428695,0.068667,-0.043812
TCGA-3C-AALJ-01,1,62.0,0.0,1.082808,-4.623726,-0.531035,0.484028,-0.183678,-0.91901,0.261106,...,-0.748878,-0.656117,-0.216733,0.21081,0.101023,0.660514,1.295073,0.915105,0.168567,0.047788
TCGA-3C-AALK-01,1,52.0,0.0,0.121608,-2.881526,-0.531035,0.179128,0.039222,-0.45491,0.180306,...,0.415422,-0.178417,-0.211233,-1.28139,0.108023,1.132814,0.496773,0.240105,3.099767,0.112888
TCGA-4H-AAAK-01,1,50.0,0.0,0.420208,-3.282726,-0.531035,-0.020972,-0.117978,-0.55781,-0.173794,...,0.685222,-0.698717,-0.484233,-1.28139,-0.223577,1.222714,0.225573,-0.301995,0.200067,-0.080212


In [20]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1215 samples.


In [21]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Epilepsy', the least common label is '0' with 113 occurrences. This represents 9.30% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

Quartiles for 'Age':
  25%: 48.0
  50% (Median): 58.0
  75%: 67.0
Min: 26.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 13 occurrences. This represents 1.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [22]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [23]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Haoyang/Epilepsy/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [7]:
dataset = 'GEO'
trait_subdir = "Underweight"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE130563',
 'GSE119483',
 'GSE57802',
 'GSE84954',
 'GSE163901',
 'GSE10135',
 'GSE85017',
 'GSE163902',
 'GSE50982',
 'GSE163903',
 'GSE131835']

Repeat the below steps for all the accession numbers

In [9]:
cohort = accession_num = "GSE130563"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/wjt/Desktop/is_data/GEO/Underweight/GSE130563/GSE130563_family.soft.gz',
 '/Users/wjt/Desktop/is_data/GEO/Underweight/GSE130563/GSE130563_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [10]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Skeletal Muscle Fibrosis in Pancreatic Cancer Patients with Respect to Survival"
!Series_summary	"Skeletal muscle wasting is a devastating consequence of cancer that may be responsible for nearly 30% of cancer-related deaths. In addition to muscle atrophy, we have identified significant muscle fiber damage and replacement of muscle with fibrotic tissue in rectus abdominis muscle biopsies from cachectic pancreatic ductal adenocarcinoma (PDAC) patients that associates with poor survival. Transcriptional profiling of muscle harvested from these same patients supported these findings by identifying gene clusters related to wounding, inflammation and cellular response to TGF-B upregulated in cachectic PDAC patients compared with non-cancer controls."
!Series_summary	"In this dataset, we include the expression data obtained from rectus abdominis muscle biopsies fron non-cancer controls patients undergoing abdominal surgery for benign reasons and from PDAC patients undergoing t

In [11]:
clinical_data.head()

,!Sample_geo_accession,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
0,!Sample_characteristics_ch1,diagnosis: mucinous cystadenoma,diagnosis: squamoid cyst,diagnosis: IPMN,diagnosis: bile duct injury,diagnosis: choledocholithiasis s/p cholecystec...,diagnosis: common bile duct stricture,diagnosis: cholecystitis,diagnosis: bile duct injury and acute pancreat...,diagnosis: open cholecystectomy,...,diagnosis: pancreatic ductal adenocarcinoma,diagnosis: pancreatic ductal adenocarcinoma,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis,diagnosis: chronic pancreatitis
1,!Sample_characteristics_ch1,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,...,Sex: F,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,Sex: F,Sex: F,Sex: M,Sex: M
2,!Sample_characteristics_ch1,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,...,tnm: pT3N1M0,tnm: pT3N0M0,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a,tnm: n/a
3,!Sample_characteristics_ch1,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 4,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 3,...,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: 0,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...,bw loss in 6 months prior to surgery: n.d. (no...
4,!Sample_characteristics_ch1,age: 33,age: 68,age: 73,age: 49,age: 78,age: 57,age: 55,age: 50,age: 47,...,age: 59,age: 74,age: 30,age: 51,age: 55,age: 55,age: 45,age: 58,age: 50,age: 54


In [12]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['diagnosis: mucinous cystadenoma',
  'diagnosis: squamoid cyst',
  'diagnosis: IPMN',
  'diagnosis: bile duct injury',
  'diagnosis: choledocholithiasis s/p cholecystectomy',
  'diagnosis: common bile duct stricture',
  'diagnosis: cholecystitis',
  'diagnosis: bile duct injury and acute pancreatic necrosis',
  'diagnosis: open cholecystectomy',
  'diagnosis: acute pancreatic necrosis',
  'diagnosis: sclerosing cholangitis',
  'diagnosis: stricture of choledochojejunostomy',
  'diagnosis: common bile duct injury',
  'diagnosis: choledochal cyst',
  'diagnosis: pancreatic ductal adenocarcinoma',
  'diagnosis: chronic pancreatitis'],
 1: ['Sex: F', 'Sex: M'],
 2: ['tnm: n/a',
  'tnm: pT3N1M0',
  'tnm: pT3N0M0',
  'tnm: Stage IV',
  'tnm: pT4N1M0',
  'tnm: pT1N0M0',
  'tnm: pT2N1M0'],
 3: ['bw loss in 6 months prior to surgery: 0',
  'bw loss in 6 months prior to surgery: 4',
  'bw loss in 6 months prior to surgery: 3',
  'bw loss in 6 months prior to surgery: 6.5',
  'bw loss in 6 m

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [16]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Obstructive sleep apnea\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Obstructive sleep apnea\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is re

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [14]:
is_gene_availabe = True
trait_row = 0
age_row = 4
gender_row = 1

trait_type = 'binary'

In [15]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [16]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'diagnosis: mucinous cystadenoma.
    """
    if tissue_type == 'diagnosis: mucinous cystadenoma':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: F':
        return 0
    elif gender_string == 'Sex: M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [17]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,GSM3743564,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
Underweight,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,33,68,73,49,78,57,55,50,47,63,...,59,74,30,51,55,55,45,58,50,54
Gender,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,0,1,1


In [18]:
### Genetic data preprocessing and final filtering

In [19]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM3743555,GSM3743556,GSM3743557,GSM3743558,GSM3743559,GSM3743560,GSM3743561,GSM3743562,GSM3743563,GSM3743564,...,GSM3743591,GSM3743592,GSM3743593,GSM3743594,GSM3743595,GSM3743596,GSM3743597,GSM3743598,GSM3743599,GSM3743600
ID,,,,,,,,,,,,,,,,,,,,,
100009613_at,2.484554,2.170547,2.240293,2.017631,2.322433,2.320112,2.274520,1.976751,2.118517,2.305105,...,1.998159,2.574119,2.171907,2.048359,2.141913,2.141388,2.387593,1.884169,1.866739,2.190988
100009676_at,3.746875,3.229653,3.505413,3.202440,3.726104,2.772018,3.349980,3.028347,3.041839,3.240944,...,3.200839,2.428775,3.050473,2.940380,3.182597,2.983551,3.162175,3.118802,2.762642,3.217956
10000_at,4.383380,4.920306,4.963512,4.992320,5.066081,4.510955,5.113312,4.481963,4.806586,5.168752,...,4.880312,5.219858,5.391569,5.379981,5.002947,4.674068,4.673622,5.638959,4.856591,5.245355
10001_at,5.822047,5.975492,5.852250,6.243071,6.405402,6.347597,6.168092,6.317435,5.907259,6.213130,...,6.260879,5.486075,6.186902,6.096248,5.928020,6.029681,6.109818,6.056322,6.086313,5.975695
10002_at,1.883013,2.279264,2.570026,2.095891,1.950774,1.904020,2.093506,2.002870,2.585459,2.275075,...,2.015685,2.353116,1.956116,2.378803,1.884763,1.839770,2.079452,1.902106,1.917236,2.439328


In [20]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['100009613_at',
 '100009676_at',
 '10000_at',
 '10001_at',
 '10002_at',
 '100033413_at',
 '100033422_at',
 '100033423_at',
 '100033424_at',
 '100033425_at',
 '100033426_at',
 '100033427_at',
 '100033428_at',
 '100033430_at',
 '100033431_at',
 '100033432_at',
 '100033434_at',
 '100033435_at',
 '100033436_at',
 '100033437_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [21]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at', '100033422_at', '100033423_at', '100033424_at', '100033425_at', '100033426_at', '100033427_at', '100033428_at', '100033430_at', '100033431_at', '100033432_at', '100033434_at', '100033435_at', '100033436_at', '100033437_at']\n"

If not required, jump directly to the gene normalization step

In [22]:
requires_gene_mapping = True

In [23]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000']}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [24]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ORF': ['1', '10', '100', '1000', '10000']}
    


In [25]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'ORF'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [26]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [29]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [30]:
print(f"The merged dataset contains {len(merged_data)} samples.")
merged_data

The merged dataset contains 48 samples.


,Obstructive sleep apnea,Age,Gender,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
GSM5717479,1.0,58.0,1.0,0.564183,0.805036,0.764749,0.802062,0.373054,0.738113,0.776844,...,0.197099,0.118462,0.327011,0.365091,0.320941,0.567943,0.521306,0.359458,0.499577,0.199448
GSM5717480,1.0,47.0,1.0,0.625571,0.810280,0.780828,0.810802,0.380973,0.715563,0.770553,...,0.186371,0.094386,0.284797,0.391011,0.321053,0.554167,0.526003,0.331681,0.494205,0.176213
GSM5717481,1.0,33.0,1.0,0.545399,0.814929,0.770501,0.816325,0.384366,0.695083,0.739911,...,0.190216,0.104534,0.311023,0.396903,0.336653,0.568726,0.516668,0.344406,0.499049,0.181043
GSM5717482,1.0,41.0,1.0,0.606818,0.780134,0.794368,0.784505,0.377553,0.680227,0.751351,...,0.190320,0.106124,0.328337,0.389400,0.325795,0.578876,0.522967,0.362627,0.507475,0.182782
GSM5717483,1.0,45.0,1.0,0.617074,0.810361,0.777432,0.817517,0.381875,0.714298,0.771476,...,0.188320,0.087550,0.301792,0.379823,0.325478,0.548225,0.525753,0.337893,0.497991,0.169523
GSM5717484,1.0,47.0,1.0,0.630901,0.799217,0.770812,0.782668,0.377651,0.687498,0.746058,...,0.182088,0.091448,0.298949,0.396714,0.330842,0.561695,0.517238,0.337792,0.492311,0.179240
GSM5717485,1.0,40.0,1.0,0.573520,0.811120,0.777460,0.777875,0.376042,0.700447,0.773809,...,0.193197,0.098966,0.336880,0.445775,0.329461,0.570147,0.513723,0.365833,0.503252,0.185596
GSM5717486,1.0,28.0,1.0,0.589640,0.820866,0.792548,0.780909,0.376011,0.742172,0.799819,...,0.196509,0.115313,0.296931,0.396388,0.321002,0.570476,0.511395,0.360828,0.510382,0.194992
GSM5717487,1.0,46.0,1.0,0.580706,0.834600,0.777558,0.784991,0.382957,0.747320,0.795676,...,0.193843,0.098934,0.284718,0.405338,0.326023,0.563181,0.530072,0.363634,0.503618,0.183152
GSM5717488,1.0,57.0,1.0,0.567496,0.790446,0.776891,0.769835,0.381954,0.688085,0.762448,...,0.189139,0.089909,0.310228,0.399793,0.322123,0.586155,0.510918,0.323674,0.489876,0.177189


In [31]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Obstructive sleep apnea', the least common label is '0.0' with 16 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Obstructive sleep apnea' in this dataset is fine.

Quartiles for 'Age':
  25%: 36.0
  50% (Median): 41.5
  75%: 47.75
Min: 28.0
Max: 58.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 2 occurrences. This represents 4.17% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



False

In [32]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: /Users/wjt/Desktop/is_data/v1/Lisa/Obstructive-sleep-apnea/cohort_info.json


In [33]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [34]:
from utils import *

In [35]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE190164'

In [36]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE190164,True,True,False,True,False,48,


In [37]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [38]:
merged_data.head()

,Obstructive sleep apnea,Age,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1.0,58.0,0.564183,0.805036,0.764749,0.802062,0.373054,0.738113,0.776844,0.193147,...,0.197099,0.118462,0.327011,0.365091,0.320941,0.567943,0.521306,0.359458,0.499577,0.199448
1,1.0,47.0,0.625571,0.810280,0.780828,0.810802,0.380973,0.715563,0.770553,0.176388,...,0.186371,0.094386,0.284797,0.391011,0.321053,0.554167,0.526003,0.331681,0.494205,0.176213
2,1.0,33.0,0.545399,0.814929,0.770501,0.816325,0.384366,0.695083,0.739911,0.182338,...,0.190216,0.104534,0.311022,0.396903,0.336653,0.568726,0.516668,0.344406,0.499049,0.181043
3,1.0,41.0,0.606818,0.780134,0.794368,0.784505,0.377553,0.680227,0.751351,0.183316,...,0.190320,0.106124,0.328337,0.389400,0.325795,0.578876,0.522967,0.362627,0.507475,0.182782
4,1.0,45.0,0.617074,0.810361,0.777432,0.817517,0.381875,0.714298,0.771476,0.175075,...,0.188320,0.087550,0.301792,0.379823,0.325478,0.548225,0.525753,0.337893,0.497991,0.169523


In [39]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [41]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [42]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [43]:
if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [44]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 53.33% ± 10.89%


In [45]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 73.33% ± 13.33%


In [46]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [47]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.1290
Found 40 genes with non-zero coefficients associated with the trait 'Obstructive sleep apnea' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable   Coefficient  Absolute Coefficient
  SAMSN1  5.443527e+00          5.443527e+00
 FILIP1L  5.304410e+00          5.304410e+00
   CRTAM -5.137245e+00          5.137245e+00
   FASLG -4.965498e+00          4.965498e+00
   ANKAR -4.857033e+00          4.857033e+00
  RNASEL -4.175238e+00          4.175238e+00
  NFE2L3  3.666362e+00          3.666362e+00
  TAS2R3 -3.645469e+00          3.645469e+00
   CMTM1  3.295981e+00          3.295981e+00
  NHLRC4  3.070194e+00          3.070194e+00
  BTN2A1 -3.053355e+00          3.053355e+00
SCARNA23 -2.958934e+00          2.958934e+00
C10orf62 -2.612233e+00          2.612233e+00
   THBS4  2.388604e+00          2.388604e+00
    TYMP  2.386668e+00          2.386668e+0

/Users/wjt/Desktop/TAIS_2024SP/Lisa/utils.py:486: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [48]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.1290
Found 2 significant genes associated with the trait 'Obstructive sleep apnea' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient  p_value  corrected_p_value
  PDLIM4    -0.335979 0.000002           0.027557
   GSTM5    -0.379023 0.000005           0.045641


/Users/wjt/Desktop/TAIS_2024SP/Lisa/utils.py:495: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
